<a href="https://colab.research.google.com/github/lmolinario/ML_Sec_project/blob/main/MLSEC_FNM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Machine Learning Security Project - Project 1**
###**Student: Lello Molinario (70/90/00369)**

**Instructions**

Re-evaluate 5 RobustBench models with another attack algorithm (e.g. FMN) and identify samples for which one attack works and the other doesn't. Explain the results - i.e., provide some motivations on why one of the attacks did not work properly, while the other did.


**Execution**

In order to re-evaluate 5 RobustBench models I chose "FNM" as the attack algorithm.

# **Step 1: Installing libraries and resolving incompatibilities**

In [ ]:
!pip install git+https://github.com/RobustBench/robustbench.git

  Cloning https://github.com/RobustBench/robustbench.git to /tmp/pip-req-build-ck1uecla
  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench.git /tmp/pip-req-build-ck1uecla
  Resolved https://github.com/RobustBench/robustbench.git to commit 46a91f44524133b2cd8f721ec7e73ecb63f17fc8
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fra31/auto-attack.git (to revision a39220048b3c9f2cca9a4d3a54604793c68eca7e) to /tmp/pip-install-b6ch2svg/autoattack_585f535008484cc6ac82c287431dae92
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack.git /tmp/pip-install-b6ch2svg/autoattack_585f535008484cc6ac82c287431dae92
  Running command git rev-parse -q --verify 'sha^a39220048b3c9f2cca9a4d3a54604793c68eca7e'
  Running command git fetch -q https://github.com/fra31/auto-attack.git a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Resolved https://github.com/fra31/auto-attack.git to commit a39220048b3c9f2cca9a4

In [ ]:
!pip install secml foolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.0/431.0 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.3 MB/s eta 0:00:00
  Created wheel for matplotlib: filename=matplotlib-3.0.3-cp311-cp311-linux_x86_64.whl size=11723811 sha256=8297739f2e5454a8cadb1bf0486b1232442800159abe195efb0a7bda9a515ac7
  Stored in directory: /root/.cache/pip/wheels/e7/99/58/7ddb91918cb838a6f4099c834d5bf43c1262ab8cded5755817
Successfully built matplotlib
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplot

In [ ]:
import robustbench
import secml
import foolbox

print(f"RobustBench version: {robustbench.__name__}")
print(f"SecML version: {secml.__version__}")
print(f"Foolbox version: {foolbox.__version__}")

RobustBench version: robustbench
SecML version: 0.13.post1
Foolbox version: 3.3.4


In [ ]:
#import secml
#import foolbox as fb
#import robustbench


import os
import gc
import torch
import numpy as np

import matplotlib.pyplot as plt

from secml.array import CArray
from secml.ml import CClassifierPyTorch
from secml.ml.peval.metrics import CMetricAccuracy
#from secml.adv.attacks.evasion import CAttackEvasionFoolbox
from secml.data.loader import CDataLoaderCIFAR10
from secml.ml.features.normalization import CNormalizerMinMax
from secml.figure import CFigure
from secml.explanation import \
    CExplainerGradient, CExplainerGradientInput, CExplainerIntegratedGradients
from secml.utils import fm
from secml import settings
from secml.ml.classifiers.loss import CSoftmax

from robustbench.utils import load_model

from torch import nn

2025-01-27 15:36:58,284 - secml.settings - INFO - New `SECML_HOME_DIR` created: /root/secml-data
2025-01-27 15:36:58,284 - secml.settings - INFO - New `SECML_HOME_DIR` created: /root/secml-data


INFO:secml.settings:New `SECML_HOME_DIR` created: /root/secml-data


2025-01-27 15:36:58,288 - secml.settings - INFO - Default configuration file copied to: /root/secml-data/secml.conf
2025-01-27 15:36:58,288 - secml.settings - INFO - Default configuration file copied to: /root/secml-data/secml.conf


INFO:secml.settings:Default configuration file copied to: /root/secml-data/secml.conf


2025-01-27 15:36:58,292 - secml.settings - INFO - New `SECML_DS_DIR` created: /root/secml-data/datasets
2025-01-27 15:36:58,292 - secml.settings - INFO - New `SECML_DS_DIR` created: /root/secml-data/datasets


...

INFO:secml.settings:New `SECML_EXP_DIR` created: /root/secml-data/experiments


2025-01-27 15:36:58,311 - secml.settings - INFO - New `SECML_LOGS_DIR` created: /root/secml-data/logs
2025-01-27 15:36:58,311 - secml.settings - INFO - New `SECML_LOGS_DIR` created: /root/secml-data/logs


INFO:secml.settings:New `SECML_LOGS_DIR` created: /root/secml-data/logs


2025-01-27 15:36:58,317 - secml.settings - INFO - New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data
2025-01-27 15:36:58,317 - secml.settings - INFO - New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data


INFO:secml.settings:New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data


## Global Variables
Contains definition of global variables

In [19]:
input_shape    = (3, 32, 32)
model_names    = [
    "Ding2020MMA",
    "Wong2020Fast",
    "Andriushchenko2020Understanding",
    "Sitawarin2020Improving",
"Debenedetti2022Light_XCiT-L12"
]
n_samples      = 64
dataset_labels = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

## Loading models

Loads five models from robustbench. We have chosen the following models:
*   1 - Ding2020MMA
*   2 - Wong2020Fast
*   3 - Andriushchenko2020Understanding
*   4 - Sitawarin2020Improving
*   5 - Gowal2020Uncovering_70_16_extra

CIFAR-10
Linf, eps=8/255


In [20]:
def load_model(model_name):
    """
    Load a single model from Robustbench and
    wrap it into a CClassifier
    """
    model = robustbench.utils.load_model(
        model_name=model_name,
        dataset='cifar10',
        threat_model='Linf',
    )

    clf  = CClassifierPyTorch(
        model,
        input_shape=input_shape,
        pretrained=True,
        pretrained_classes=CArray(list(range(10))),
        preprocess=None
    )

    return clf



models = []
for name in model_names:
    try:
        model = load_model(name)
        models.append(model)
    except Exception as e:
        print(f"Error loading model {name}: {e}")

  checkpoint = torch.load(model_path, map_location=torch.device('cpu'))

Downloading: "https://github.com/RobustBench/robustbench/releases/download/v1.1/debenedetti2022light-xcit-l-cifar10-linf.pth.tar" to /root/.cache/torch/hub/checkpoints/debenedetti2022light-xcit-l-cifar10-linf.pth.tar


## Loading  CIFAR-10

Loads 64 samples from CIFAR-10 dataset with shape (3, 32, 32)

In [17]:
tr, ts      = CDataLoaderCIFAR10().load()
normalizer  = CNormalizerMinMax().fit(tr.X)

ts          = ts[:n_samples, :]

# Normalize test samples
ts.X = normalizer.transform(ts.X)

## Fast-Minimum-Norm (FMN) attack

Computes the accuracy of the models, just to confirm that it is working properly.

In [21]:
# Calcolo delle predizioni e accuratezza dei modelli
metric = CMetricAccuracy()
models_preds = [clf.predict(ts.X) for clf in models]
accuracies = [metric.performance_score(y_true=ts.Y, y_pred=y_pred) for y_pred in models_preds]

print("-" * 90)
# Stampa delle accuratezze
for idx in range(len(model_names)):
    print(f"Model name: {model_names[idx]:<40} - Clean model accuracy: {(accuracies[idx] * 100):.2f} %")
print("-" * 90)

------------------------------------------------------------------------------------------
Model name: Ding2020MMA                              - Clean model accuracy: 84.38 %
Model name: Wong2020Fast                             - Clean model accuracy: 84.38 %
Model name: Andriushchenko2020Understanding          - Clean model accuracy: 78.12 %
Model name: Sitawarin2020Improving                   - Clean model accuracy: 82.81 %
Model name: Debenedetti2022Light_XCiT-L12            - Clean model accuracy: 90.62 %
------------------------------------------------------------------------------------------
